<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# NLP Using the Twitter API: Guided Lab

_Authors: Dave Yerrington (SF)_

---


<img src="https://snag.gy/RNAEgP.jpg" width="600">

### Can we correctly identify which of these two old men tweeted what?

> *Note: this lab is intended to be a guided lab until the independent practice questions.*


## Goals
---

We are going to attempt to classify whether a tweet comes from Trump or Sanders.  This lab involves multiple steps:
- Create a developer account on Twitter
- Create a method to pull a list of tweets from the Twitter API
- Perform proper preprocessing on our text
- Engineer sentiment feature in our dataset using TextBlob
- Explore supervised classification techniques


## Twitter API Developer Registration
---

If you haven't registered a Twitter account yet, this is a requirement in order to have a "developer" account.

[Twitter Rest API](https://dev.twitter.com/rest/public)



## Create an "App"

---

![](https://snag.gy/HPBQbJ.jpg)

Go to Twitter and register an "app" [apps.twitter.com](https://apps.twitter.com/).

> **Note**: For the required website field you can put a placeholder.

After you set up our app, you will only need to reference the cooresponding keys Twitter generates for our app.  These are the keys that we will use with our application to communicate with the Twitter API.

## Install Python Twitter API library

---

Someone was nice enough to build a Python libary for us. It makes pulling tweets simple: we only need to plug in our keys and start collecting data. The library we will be using is provided by [Python Twitter Tools](http://mike.verdone.ca/twitter/).

To install it, just run the next frame (there is no conda package).

## Some Boring Twitter Rules
---

**Twitter notifies you they will rate limit your requests:**

>When using application-only authentication, rate limits are determined globally for the entire application. If a method allows for 15 requests per rate limit window, then it allows you to make 15 requests per window — on behalf of your application. This limit is considered completely separately from per-user limits. https://dev.twitter.com/rest/public/rate-limiting

Here's a quick overview of what Twitter says are "the rules":

![](https://snag.gy/yJ6vIH.jpg)


## About those Keys: OAuth Review
---

![](https://g.twimg.com/dev/documentation/image/appauth_0.png)

## What's going on here?  Take a minute..

## Our Application Keys
---

Take note of your application keys you will use to connect to Twitter and mine tweets from the official Bernie Sanders and Donald Trump twitter accounts:

![](https://snag.gy/H1djQK.jpg)

## `TweetMiner` class structure

---

The following code will get you up and running, providing connectivity to twitter. The class has the ability to make requests and can eventually transform the JSON responses into DataFrames.

This is a great example of using object-oriented Python to organize our code!

> **Note:** "request_limit" is used in this class to limit the number of tweets that are pulled per instance request.  Setting it to something lower until you've worked the bugs out of your request, and captured the data you want, is essential to avoiding the rate limit blocks.

### Twitter API key setup

Fill the information below in with the keys for your account.

- **consumer_key** - Find this in your app page under the "Keys and Access Tokens"
- **consumer_secret** - Right under **consumer_key** in the "Keys and Access Tokens" tab
- **access_token_key** - You will need to click the button to generate tokens to get this
- **access_token_secret** - Also available after you generate tokens


In [1]:
# !pip install python-twitter

In [2]:
# from twitter import *

# api = Twitter(
#     auth=OAuth(token=twitter_keys['access_token_key'], 
#                token_secret=twitter_keys['access_token_secret'], 
#                consumer_key=twitter_keys['consumer_key'], 
#                consumer_secret=twitter_keys['consumer_secret']))

In [2]:
print(twitter.__version__)

3.4.2


In [1]:
import twitter, re, datetime, pandas as pd

twitter_keys = {
    'consumer_key':        'lA6UQplM5sxIAIr83ueUl9sgE',
    'consumer_secret':     'f94t4BD6Vj7aCVkX1qAIVwsP4x69J2vXvm61lTIuwb9GfmdsuP',
    'access_token_key':    '1203210464-dYE1FvoUx1GjVcoyok3U1brWDpELBJEcSNGC1OC',
    'access_token_secret': 'kuVli1j010RPdYTboz9iIyp8QxX6PqRApdi49baLqBgzo'
}

api = twitter.Api(
    consumer_key         =   twitter_keys['consumer_key'],
    consumer_secret      =   twitter_keys['consumer_secret'],
    access_token_key     =   twitter_keys['access_token_key'],
    access_token_secret  =   twitter_keys['access_token_secret']
)


In [4]:
class TweetMiner(object):

    result_limit    =   20    
    api             =   False
    data            =   []
    
    twitter_keys = {
        'consumer_key':        'KmN03M1X1pImZ43sqdIu4yfnE',
        'consumer_secret':     'ePlIrIX5VXbZnO7DBu1RbFlw5lOai9dQr9n5TZb6vxnIdrr5Fz',
        'access_token_key':    '185036086-Q7K5IjuSoQZJwSIqD0wyHf6t62iPKatmfaPkriAM',
        'access_token_secret': 'cYpQz3xWHQbLplOj8iSeiNSOmMcsTOXmWcKMrJ9buLj5d'
    }
    
    def __init__(self, keys_dict, api, result_limit = 20):
        
        self.api = api
        self.twitter_keys = keys_dict
        
        self.result_limit = result_limit
        

    def mine_user_tweets(self, user="dyerrington", mine_rewteets=False, max_pages=5):

        data           =  []
        last_tweet_id  =  False
        page           =  1
        
        while page <= max_pages:
            
            if last_tweet_id:
                statuses   =   self.api.GetUserTimeline(screen_name=user, count=self.result_limit, max_id=last_tweet_id - 1)        
            else:
                statuses   =   self.api.GetUserTimeline(screen_name=user, count=self.result_limit)
                
            for item in statuses:

                mined = {
                    'tweet_id':        item.id,
                    'handle':          item.user.name,
                    'retweet_count':   item.retweet_count,
                    'text':            item.text,
                    'mined_at':        datetime.datetime.now(),
                    'created_at':      item.created_at,
                }
                
                last_tweet_id = item.id
                data.append(mined)
                
            page += 1
            
        return data

## Instantiate the class
---

Make sure you pass the keys dictionary and the api as arguments.

**Check:** call the object's `mine_user_tweets()` method, providing a user to pull the tweets of.

In [5]:
miner = TweetMiner(twitter_keys, api, result_limit=2)

In [6]:
sanders = miner.mine_user_tweets(user="berniesanders", max_pages=5)
donald = miner.mine_user_tweets(user="realDonaldTrump", max_pages=5)

In [7]:
sanders[0]

{'tweet_id': 1052365959065980928,
 'handle': 'Bernie Sanders',
 'retweet_count': 1305,
 'text': 'In the midterm election of 2014, the United States had the lowest voter turnout since World War II. If we increase… https://t.co/qPshpETduY',
 'mined_at': datetime.datetime(2018, 10, 17, 13, 33, 56, 740504),
 'created_at': 'Wed Oct 17 01:09:31 +0000 2018'}

In [8]:
donald[0]

{'tweet_id': 1052388580734263297,
 'handle': 'Donald J. Trump',
 'retweet_count': 6689,
 'text': 'Stock Market up 548 points today. Also, GREAT jobs numbers!',
 'mined_at': datetime.datetime(2018, 10, 17, 13, 33, 59, 398357),
 'created_at': 'Wed Oct 17 02:39:24 +0000 2018'}

### Convert the tweet ouputs to a pandas DataFrame

> *Hint: this is as easy as passing it to the DataFrame constructor!*

In [9]:
pd.DataFrame(sanders).head()

,created_at,handle,mined_at,retweet_count,text,tweet_id
0,Wed Oct 17 01:09:31 +0000 2018,Bernie Sanders,2018-10-17 13:33:56.740504,1305,"In the midterm election of 2014, the United St...",1052365959065980928
1,Wed Oct 17 00:34:22 +0000 2018,Bernie Sanders,2018-10-17 13:33:56.740517,70,"RT @IronStache: Join @BernieSanders, the @WIwf...",1052357111999393793
2,Wed Oct 17 00:34:01 +0000 2018,Bernie Sanders,2018-10-17 13:33:57.252814,499,Arkansas kicked thousands of low-income people...,1052357025932431361
3,Wed Oct 17 00:05:38 +0000 2018,Bernie Sanders,2018-10-17 13:33:57.252828,240,Our job is to make it easier for people to vot...,1052349883158601728
4,Wed Oct 17 00:03:08 +0000 2018,Bernie Sanders,2018-10-17 13:33:57.968566,119,RT @gretchenwhitmer: Join us on Friday for a r...,1052349253509636096


##  Create the training data

---

Let's get our "mined" data from the Twitter API.  

1. Mine Trump tweets
- Create a tweet DataFrame
- Mine Sanders tweets
- Append the results to our DataFrame

In [10]:
# we only need to "instantiate" once.  Then we can call mine_user_tweets as much as we want.
miner = TweetMiner(twitter_keys, api, result_limit=400)
trump_tweets = miner.mine_user_tweets("realDonaldTrump")

In [11]:
trump_df = pd.DataFrame(trump_tweets)
trump_df.shape

(993, 6)

In [12]:
bernie_tweets = miner.mine_user_tweets('berniesanders')

In [13]:
bernie_df = pd.DataFrame(bernie_tweets) 
bernie_df.shape

(998, 6)

In [14]:
tweets = pd.concat([trump_df, bernie_df], axis=0)
tweets.shape

(1991, 6)

## Any interesting ngrams going on with Trump?
---

Set up a vectorizer from sklearn and fit the text of Trump's tweets with an ngram range from 2 to 4. Figure out what the most common ngrams are.

> **Note:** It's up to you whether you want to remove stopwords or not. How does keeping or removing stopwords affect the results?

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter

# We can use the TfidfVectorizer to find ngrams for us
vect = TfidfVectorizer(ngram_range=(2,4))

# Pulls all of trumps tweet text's into one giant string
summaries = "".join(trump_df['text'])
ngrams_summaries = vect.build_analyzer()(summaries)

Counter(ngrams_summaries).most_common(20)

[('https co', 744),
 ('of the', 85),
 ('in the', 49),
 ('will be', 45),
 ('to the', 38),
 ('thank you', 32),
 ('president trump', 30),
 ('the https', 30),
 ('the https co', 30),
 ('for the', 29),
 ('at the', 27),
 ('and the', 22),
 ('on the', 22),
 ('you https', 22),
 ('you https co', 22),
 ('we are', 21),
 ('our great', 21),
 ('america great', 21),
 ('great again', 21),
 ('our country', 21)]

### Look at the ngrams for Bernie Sanders

In [16]:
# We can use the TfidfVectorizer to find ngrams for us
vect = TfidfVectorizer(ngram_range=(2,4))

# Pulls all of trumps tweet text's into one giant string
summaries = "".join(bernie_df['text'])
ngrams_summaries = vect.build_analyzer()(summaries)

Counter(ngrams_summaries).most_common(20)

[('https co', 686),
 ('health care', 76),
 ('in the', 59),
 ('of the', 56),
 ('bernie sanders', 54),
 ('to be', 52),
 ('to the', 49),
 ('for the', 39),
 ('we must', 37),
 ('for all', 36),
 ('it is', 34),
 ('is not', 32),
 ('this country', 30),
 ('we are', 28),
 ('on the', 28),
 ('we can', 26),
 ('proud to', 26),
 ('thank you', 26),
 ('that the', 25),
 ('in this', 24)]

## Processing the tweets and building a model

---

To do classfication we will need to convert the tweets into a set of features.

**You will need to:**
- Vectorize input text data.
- Intialize a model (try Logistic regression).
- Train / Predict / cross-validate.
- Evaluate the performance of the model.

> **Bonus:** you may have noticed that there are website links in the tweets. What additional preprocessing steps can you do before building the model?


In [17]:
# ! pip install textacy

In [18]:
# BONUS
# Using the textacy package to do some more comprehensive preprocessing
# http://textacy.readthedocs.io/en/latest/
from textacy.preprocess import preprocess_text
import numpy as np

tweet_text = tweets['text'].values
clean_text = [preprocess_text(x, fix_unicode=True, lowercase=True, transliterate=False,
                              no_urls=True, no_emails=True, no_phone_numbers=True, no_currency_symbols=True,
                              no_punct=True, no_accents=True)
              for x in tweet_text]

In [19]:
tweet_text[0:3]

array(['Stock Market up 548 points today. Also, GREAT jobs numbers!',
       'Elizabeth Warren is being hammered, even by the Left. Her false claim of Indian heritage is only selling to VERY LOW I.Q. individuals!',
       'Anybody entering the United States illegally will be arrested and detained, prior to being sent back to their country!'],
      dtype=object)

In [20]:
clean_text[0:3]

['stock market up 548 points today also great jobs numbers',
 'elizabeth warren is being hammered even by the left her false claim of indian heritage is only selling to very low i q individuals',
 'anybody entering the united states illegally will be arrested and detained prior to being sent back to their country']

In [21]:
# target is the handle.
# make trump 1 and sanders 0
y = tweets['handle'].map(lambda x: 1 if x == 'Donald J. Trump' else 0).values
np.mean(y)

0.49874434957307884

In [22]:
from sklearn.linear_model import LogisticRegression

# Preprocess our text data to Tfidf
tfv = TfidfVectorizer(ngram_range=(1,4), max_features=2000)
X = tfv.fit_transform(clean_text).todense()
X.shape

(1991, 2000)

In [23]:
# cross-validate the accuracy:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(LogisticRegression(), X, y, cv=10)

import numpy as np
print(accuracies)
print(np.mean(accuracies))

# Setup logistic regression (or try another classification method here)
estimator = LogisticRegression()
estimator.fit(X, y)


[0.9        0.86       0.885      0.8241206  0.92964824 0.87939698
 0.86934673 0.79396985 0.92424242 0.8989899 ]
0.8764714735292625


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [24]:
# Very good accuracy considering the baseline is 50%

## Check the predicted probability for a random Sanders and Trump tweet
---

Below are provided a couple of tweets from both Sanders and Trump. I'm sure you can figure out on your own which one is which.

Estimate the predicted probability of being trump for the two tweets.

In [25]:
# Prep our source as TfIdf vectors
source_test = [
    "Demanding that the wealthy and the powerful start paying their fair share of taxes that's exactly what the American people want.",
    "Crooked Hillary is spending tremendous amounts of Wall Street money on false ads against me. She is a very dishonest person!"
]

############
# NOTE:  Do not re-initialize the tfidf vectorizor or the feature space willbe overwritten and
# hence your transform will not match the number of features you trained your model on.
#
# This is why you only need to "transform" since you already "fit" previously
#
####

Xtest = tfv.transform(source_test)

# Predict using previously trained logist regression `estimator`
estimator.predict_proba(Xtest)


array([[0.76616939, 0.23383061],
       [0.2802059 , 0.7197941 ]])

In [26]:
# The 1st column is probability of being Bernie, and 2nd Trump. The classifier is getting it right.

## Independent practice questions

---

### 1. Pull tweets for some new users.

Experiment with using more data.  The API will not like it if you blow through their limits - be careful.  Try to grab only what you need one time, then work on the copy of the objects that are returned.  

> Read the documentation about rate limits and see if you can get enough without hitting the rate limit.  Are there any options available in the API to avoid such a problem?

**Pull tweets for more than two different users of your choice.**

In [27]:
# We deviate from trump / sanders using student tweets here 
# to illustrate the NLP pipeine with twitter data

twitter_handles = ["dril", "LaziestCanine",'ch000ch']
tweets = {}

for twitter_handle in twitter_handles:
    print("Mining tweets for: ", twitter_handle)
    miner = TweetMiner(twitter_keys, api, result_limit=500)
    tweets[twitter_handle] = miner.mine_user_tweets(user=twitter_handle, max_pages=10)


Mining tweets for:  dril
Mining tweets for:  LaziestCanine
Mining tweets for:  ch000ch


In [28]:
multi = pd.DataFrame(tweets['dril'])
multi = multi.append(pd.DataFrame(tweets['LaziestCanine']))
multi = multi.append(pd.DataFrame(tweets['ch000ch']))
multi.shape

(5985, 6)

In [29]:
multi.handle.value_counts()

Lazy dog    1997
chuuch      1996
wint        1992
Name: handle, dtype: int64

### 2. Build a multi-class classification model to distinguish between the users.

Try a new type of model than we used before.

In [30]:
tweet_text = multi['text'].values
clean_text = [preprocess_text(x, fix_unicode=True, lowercase=True, transliterate=False,
                              no_urls=True, no_emails=True, no_phone_numbers=True, no_currency_symbols=True,
                              no_punct=True, no_accents=True)
              for x in tweet_text]

y = multi['handle'].map(lambda x: 0 if x == 'wint' else 1 if x == 'Lazy dog' else 2).values

In [31]:
tfv = TfidfVectorizer(ngram_range=(1,3), max_features=2500)
X = tfv.fit_transform(clean_text)

In [32]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [33]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

rf = RandomForestClassifier(n_estimators=250, verbose=1)
knn = KNeighborsClassifier(n_neighbors=7)

rf.fit(X_train, y_train)
knn.fit(X_train, y_train)

/Users/edoardo/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:    5.6s finished


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=7, p=2,
           weights='uniform')

In [34]:
# Random forest score:
print('RF:', rf.score(X_test, y_test))
print('KNN:', knn.score(X_test, y_test))

[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:    0.4s finished


RF: 0.7299554565701559
KNN: 0.41035634743875277


In [35]:
# Baseline score:
multi.handle.value_counts()/multi.shape[0]

Lazy dog    0.333668
chuuch      0.333500
wint        0.332832
Name: handle, dtype: float64

In [36]:
rf_yhat = knn.predict(X_test)

### 3. Make a confusion matrix and classification report.

In [37]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(y_test, rf_yhat))

             precision    recall  f1-score   support

          0       0.55      0.16      0.24       630
          1       0.37      0.71      0.49       585
          2       0.44      0.38      0.41       581

avg / total       0.46      0.41      0.38      1796



In [38]:
# Confusion Matrix
print(confusion_matrix(y_test, rf_yhat))

[[ 98 370 162]
 [ 52 417 116]
 [ 28 331 222]]


### 4. What is the most and least "distinctive" tweets for each user?

To find this, identify the tweet that has the highest (correct) predicted probability of being that user's tweet for each user.

In [39]:
rf.fit(X, y)
pp = rf.predict_proba(X)

[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:    8.8s finished
[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:    1.1s finished


In [40]:
pp[0:5]

array([[0.828     , 0.048     , 0.124     ],
       [0.72107338, 0.16701928, 0.11190734],
       [0.76      , 0.016     , 0.224     ],
       [0.72      , 0.184     , 0.096     ],
       [0.748     , 0.172     , 0.08      ]])

In [41]:
pp = pd.DataFrame(pp, columns=['dril_pp', 'laziestcanine_pp', 'ch000ch_pp'])

In [42]:
multi.shape, pp.shape

((5985, 6), (5985, 3))

In [43]:
tweets_pp = pd.concat([multi.reset_index(), pp.reset_index()], axis=1)
tweets_pp.head(2)

,index,created_at,handle,mined_at,retweet_count,text,tweet_id,index,dril_pp,laziestcanine_pp,ch000ch_pp
0,0,Tue Oct 16 18:44:58 +0000 2018,wint,2018-10-17 13:34:18.559976,1659,wondering how i learned all the different info...,1052269184409690113,0,0.828000,0.048000,0.124000
1,1,Tue Oct 16 18:17:19 +0000 2018,wint,2018-10-17 13:34:18.559986,853,#WorldFoodDay food sucks. food sucks,1052262228034969600,1,0.721073,0.167019,0.111907


In [44]:
print('Most dril:', tweets_pp[tweets_pp.handle == 'wint'].sort_values('dril_pp', ascending=False).text.values[0])
print('Least dril:', tweets_pp[tweets_pp.handle == 'wint'].sort_values('dril_pp', ascending=True).text.values[0])

Most dril: @coolgothsweater shut htte fuck up
Least dril: @hhgregg 696969696969


In [45]:
print('Most LaziestCanine:', tweets_pp[tweets_pp.handle == 'Lazy dog'].sort_values('laziestcanine_pp', ascending=False).text.values[0])
print('Least LaziestCanine:', tweets_pp[tweets_pp.handle == 'Lazy dog'].sort_values('laziestcanine_pp', ascending=True).text.values[0])

Most LaziestCanine: my body is my journal and my tattoos are my story https://t.co/k4F1rUFzn5
Least LaziestCanine: RT @LameAsChris: https://t.co/IHvdeTsPya


In [46]:
print('Most chuuch:', tweets_pp[tweets_pp.handle == 'chuuch'].sort_values('ch000ch_pp', ascending=False).text.values[0])
print('Least chuuch:', tweets_pp[tweets_pp.handle == 'chuuch'].sort_values('ch000ch_pp', ascending=True).text.values[0])

Most chuuch: @BuckyIsotope cc: @FBI @CIA @JohnStamos
Least chuuch: https://t.co/CrYmbMlhag
